In [1]:
# CE 295 - Energy Systems and Control
# Final Project: Model Predictive Control and the Optimal Power Flow Problem in the IEEE 39-bus Test Feededr
# Authors (alphabetical): Carla Becker, Hannah Davalos, Jean-Luc Lupien, John Schafer, Keyi Yang
# Adapted from code provided by Prof. Daniel B. Arnold

from cvxpy import *
import data_processing as dp
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import pandas as pd

In [2]:
# Import data for IEEE 39-bus Test Feeder (plus 8 battery nodes)

# Define transformer resistance and reactance (not provided by test feeder)
xformer_r = 0.0001
xformer_x = 0.0015

# Adjacency matrix; assumes power can flow in two directions on all lines
A_df = pd.read_excel('IEEE 39 Test Bus Data/IEEE_39_bus_data.xlsx', sheet_name='A matrix')
A_df = A_df.iloc[:,1:] # remove the column labels
A    = A_df.values     # convert to a numpy array

# Resistance matrix
r_df = pd.read_excel('IEEE 39 Test Bus Data/IEEE_39_bus_data.xlsx', sheet_name='r matrix')
r_df = r_df.iloc[:,1:] # remove the column labels
r    = r_df.values     # convert to a numpy array

# Reactance matrix
x_df = pd.read_excel('IEEE 39 Test Bus Data/IEEE_39_bus_data.xlsx', sheet_name='x matrix')
x_df = x_df.iloc[:,1:] # remove the column labels
x    = x_df.values     # convert to a numpy array

# Get parents vector (matrix??)
rho_df = pd.read_excel('IEEE 39 Test Bus Data/IEEE_39_bus_data.xlsx', sheet_name='rho')
rho_df = rho_df.iloc[:,1:] # remove the column labels

# TODO how to allow for multiple parents?
future_rho = {}
for node in rho_df.columns:
    parents = np.array(rho_df[node])
    nonzero_parents = parents[parents != 0]
    key = str(node - 1) # account for indexing from 0
    future_rho[key] = nonzero_parents.tolist()

rho = rho_df.values[0,:] # TEMPORARY !! must figure out how to do multiple parents

# Number of nodes
num_nodes = A.shape[0]

# Diesel nodes
diesel_nodes = np.array([31])
diesel_nodes = diesel_nodes - 1 # adjust for indexing from 0
max_diesel_power = 10 # TODO set sensible value
diesel_cost = 100 # TODO set sensible value

# Battery energy storage nodes
BESS_nodes = np.array([40, 41, 42, 43, 44, 45, 46, 47])
BESS_nodes = BESS_nodes - 1 # adjust for indexing from 0
max_battery_power = 10 # TODO set sensible value
battery_cost = 0

# Wind nodes
wind_nodes = np.array([30, 37, 38])
wind_nodes = wind_nodes - 1 # adjust for indexing from 0
max_wind_power = 10 # TODO set sensible value
wind_cost = 0

# Solar nodes
solar_nodes = np.array([32, 33, 34, 35, 36])
solar_nodes = solar_nodes - 1 # adjust for indexing from 0
max_solar_power = 10 # TODO set sensible value
solar_cost = 0

# Consumer nodes
consumer_nodes = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 39])
consumer_nodes = consumer_nodes - 1 # adjust for indexing from 0

In [3]:
# Read in daily generation and load data
pv_directory   = 'PV Generation Data'
load_directory = 'Building Load Data'
wind_directory = 'Wind Generation Data'

# ALREADY GENERATED, takes 5 minutes to generate again so keep commented out
#dp.generate_json_from_pv_data(pv_directory) 
#dp.generate_json_from_bldg_data(load_directory) 
#dp.generate_json_from_wind_data(wind_directory) 

with open(os.path.join(pv_directory, 'pv_data.json'), 'r') as json_file:
    pv_dict = json.load(json_file)

with open(os.path.join(load_directory, 'real_data.json'), 'r') as json_file:
    real_load_dict = json.load(json_file)

with open(os.path.join(load_directory, 'reactive_data.json'), 'r') as json_file:
    reactive_load_dict = json.load(json_file)

with open(os.path.join(wind_directory, 'wind_data.json'), 'r') as json_file:
    wind_dict = json.load(json_file)

# There are 67 PV panels
# There are 27 buildings
# There are 22 wind turbines

# There are 427 days of data for each entity
# Each data has 96 data points: one measurement every 15 minutes

Reading: PV Generation Data/PowellPV.csv
Reading: PV Generation Data/CSC_BuildingPV.csv
Reading: PV Generation Data/StephenBirchPV.csv
Reading: PV Generation Data/PriceCenterA_PV.csv
Reading: PV Generation Data/EBU2_A_PV.csv
Reading: PV Generation Data/SME_SolarPV.csv
Reading: PV Generation Data/BSB_LibraryPV.csv
Reading: PV Generation Data/MayerHallPV.csv
Reading: PV Generation Data/MESOM_PV.csv
Reading: PV Generation Data/KeelingB_PV.csv
Reading: PV Generation Data/HopkinsParkingPV.csv
Reading: PV Generation Data/GarageFleetsPV.csv
Reading: PV Generation Data/BSB_BuildingPV.csv
Reading: PV Generation Data/CUP_PV.csv
Reading: PV Generation Data/LeichtagPV.csv
Reading: PV Generation Data/BioEngineeringPV.csv
Reading: PV Generation Data/EBU2_B_PV.csv
Reading: PV Generation Data/PriceCenterB_PV.csv
Reading: PV Generation Data/ElectricShopPV.csv
Reading: PV Generation Data/SDSC_PV.csv
Reading: PV Generation Data/KeelingA_PV.csv
Reading: Building Load Data/GeiselLibrary.csv
Reading: Buildi

In [ ]:
## 39 Node IEEE Test Feeder Parameters

### Node (aka Bus) Data
# l_j^P: Active power consumption [MW]
cons_real_power = real_load_dict

# l_j^Q: Reactive power consumption [MVAr]
cons_reactive_power = reactive_load_dict

# s_j,max: Maximal generating power [MW]
max_apparent_power = np.zeros(num_nodes)
max_apparent_power[diesel_nodes] = max_diesel_power
max_apparent_power[BESS_nodes]   = max_battery_power
max_apparent_power[wind_nodes]   = max_wind_power
max_apparent_power[solar_nodes]  = max_solar_power
max_apparent_power = max_apparent_power.reshape(1, num_nodes)

# c_j: Marginal generation cost [USD/MW]
c = np.zeros(num_nodes)
c[diesel_nodes] = diesel_cost
c[BESS_nodes]   = battery_cost
c[wind_nodes]   = wind_cost
c[solar_nodes]  = solar_cost
c = c.reshape(num_nodes, 1)

# V_min, V_max: Minimum and maximum nodal voltages [V]
min_voltage = 0.95
max_voltage = 1.05

# I_max_ij: Maximal line current [p.u.]
I_max = A * 0.01

### Set Data
# List of node indices
j_idx = np.arange(47) # TODO necessary?

In [ ]:
# Organize consumed real and reactive power into numpy arrays
# Only load 1 day of data (horizon is 1 day, 96 points)
num_bldgs = len(real_load_dict.keys())
cons_real_power = np.zeros((len(consumer_nodes), 96))

# For first 27 consumer nodes (only 27 buildings)
for node in range(num_bldgs):
    for b, bldg in enumerate(real_load_dict.keys()):
        if node % num_bldgs == b:
            print(real_load_dict[bldg]['0'])
            cons_real_power[node, :] = real_load_dict[bldg]['0']

# For remaining consumer nodes, repeat some of the buildings
for node in range(num_bldgs, len(consumer_nodes)):
    for b, bldg in enumerate(real_load_dict.keys()):
        if node % num_bldgs == b:
            cons_real_power[node, :] = real_load_dict[bldg]['0']

# TODO fix nan read ins for building splits!

In [ ]:
def cvx_optim(cons_real_power, cons_reactive_power, max_apparent_power, min_voltage, max_voltage, BESS_nodes, wind_nodes, solar_nodes, diesel_nodes):

    horizon   = cons_real_power.shape[0]
    num_nodes = cons_real_power.shape[1]
    renew_nodes = np.concatenate([wind_nodes, solar_nodes])
    
    # Define optimization variables for generated power
    gen_real_power       = Variable((num_nodes, horizon))
    gen_reactive_power   = Variable((num_nodes, horizon))
    gen_apparent_power   = Variable((num_nodes, horizon))

    # Define optimization variables for active line power
    line_real_power      = [Variable((num_nodes, num_nodes)) for i in range(horizon)]
    line_reactive_power  = [Variable((num_nodes, num_nodes)) for i in range(horizon)]
    line_voltages        = Variable((num_nodes, horizon))
    line_complex_current = [Variable((num_nodes, num_nodes)) for i in range(horizon)]
    sigma_A              = Variable(len(renew_nodes))

    # Define optimization variables for battery energy storage system (BESS)
    BESS_energy          = Variable((num_nodes, horizon))
    BESS_chrg_dis        = Variable((num_nodes, horizon))

    # Define robust optimization decision variables for intermittent energy sources
    y = vstack([sigma_A[0], gen_apparent_power[0]])
    for i in range(1, len(renew_nodes)):
        y = vstack([y, vstack([sigma_A[i], gen_apparent_power[i]])])

    # Robust optimization parameters
    a = np.array([-1.25, 1])
    bar_a = np.tile(a, len(renew_nodes))

    e_array = np.array((0.25, 0))
    E = np.diag(np.tile(e_array, len(renew_nodes)))

    # BESS Parameters
    timestep=.25 # assuming 15min timestep
    eta=.95 #(charging /discharging efficiency)
    energy_min=0 #minimun energy level of BESS, need to define for every node seperately?
    #100 max energy level of BESS, in kWh NEED TO SCALE
    energy_max=np.array([0, 0, 0,0, 100, 100, 0, 0, 0, 0, 0]) #placeholder, need actual nodes that have BESS, needs to have 39 nodes
    
    ramp_max= 100 / timestep #1C, can discharge/charge 1 full capacity every hour, assuming timestep is in hours
    initial_energy=np.zeros(BESS_nodes) #place holder
    #where to I specify which nodes are BESS?

    # BESS initial state
    constraints = [BESS_energy[:, 0] == initial_energy]  # initial_energy needs to be defined

    # Define objective function
    objective =  Minimize(sum(c.T @ gen_apparent_power))

    # Define constraints
    # Apparent Power Limits
    constraints += [gen_apparent_power <= max_apparent_power]

    # Boundary condition for power line flows
    constraints += [line_real_power[0][0]     == 0,
                    line_reactive_power[0][0] == 0]

    # Boundary condition for squared line current
    constraints += [line_complex_current[0][0] == 0]

    # Fix node 0 voltage to be 1 "per unit" (p.u.)
    constraints += [line_voltages[0] == 1]

    # Robust Optimization Constraints
    constraints += [bar_a @ y + norm(E @ y) <= 0]
    constraints += [sigma_A >= 0, sigma_A <= 1]

    # Loop over every time step in the horizon, loop over each nodes in each time step
    for t in range(horizon):
        for jj in [0]: # j_idx
            
            #BESS static Constraints
            constraints +=[energy_min<=BESS_energy[jj, t], BESS_energy[jj, t] <= energy_max[jj]] #every battery node has energy limits for every time

            if t < horizon - 1:
                # Ramp rate constraints
                constraints += [BESS_chrg_dis[jj, t + 1] - BESS_chrg_dis[jj, t] <= ramp_max,
                                BESS_chrg_dis[jj, t] - BESS_chrg_dis[jj, t + 1] <= ramp_max]

                # BESS energy dynamics
                constraints += [BESS_energy[jj, t + 1] == BESS_energy[jj, t] + timestep * eta * BESS_chrg_dis[jj, t]]

            # Nodal voltage limits, not dependent on t
            constraints += [min_voltage**2 <= line_voltages[jj,t]]
            constraints += [line_voltages[jj,t] <= max_voltage**2]

            # Non-negative power generation
            constraints += [gen_real_power[jj, t] >= 0]
            constraints += [gen_reactive_power[jj, t] >= 0]

            # Parent node, i = rho(j)
            ii = rho[jj][0]

            # Squared line current limits
            constraints += [line_complex_current[ii][jj] <= (I_max[ii][jj])**2] # I DONT KNOW HOW TO INDEX BY t

            # Line Power Flows
            constraints += [line_real_power[ii][jj]     == (cons_reactive_power[jj, t] - gen_apparent_power[jj,  t]) + r[ii][jj]*line_complex_current[ii][jj] + A[jj]@line_real_power[jj].T]
            constraints += [line_reactive_power[ii][jj]     == (cons_reactive_power[jj, t] - gen_apparent_power[jj,  t]) + x[ii][jj]*line_complex_current[ii][jj] + A[jj]@line_real_power[jj].T]

            # Nodal voltage
            constraints += [line_voltages[jj,t] == line_voltages[ii,t] + ((r[ii][jj])**2 + (x[ii][jj])**2)*line_complex_current[ii][jj] - 2*(r[ii][jj]*line_real_power[ii][jj]+x[ii][jj]*line_reactive_power[ii][jj])]
            
            # Squared current magnitude on lines
            constraints += [line_complex_current[ii][jj] >= quad_over_lin(vstack([line_real_power[ii][jj],line_reactive_power[ii][jj]]),line_voltages[jj,t])]

            # Compute apparent power from active & reactive power
            constraints += [norm(vstack([gen_real_power[jj,t], gen_reactive_power[jj,t]])) <= gen_apparent_power[jj,t]]

    # Define problem and solve
    prob = Problem(objective, constraints)
    prob.solve()


    # Output Results
    print(prob.status)
    print(f"Minimum Generating Cost: {prob.value} USD")

    print(" ")
    print(f"Node 30 [Diesel]: real power = {gen_real_power[30].value} MW | reactive power = {gen_reactive_power[30].value} MVAr | apparent power = {gen_apparent_power[30].value} MVA")
    
    print(" ")
    for node in renew_nodes:
        print(f"Node {node} [Renewable]: real power = {gen_real_power[node].value} MW | reactive power = {gen_reactive_power[node].value} MVAr | apparent power = {gen_apparent_power[node].value} MVA")
    
    print(" ")
    #print(f"Total active real power:     {np.sum(line_real_power)} MW consumed | {np.sum(gen_real_power)} MW generated")
    #print(f"Total active reactive power: {np.sum(line_reactive_power)} MW consumed | {np.sum(gen_reactive_power)} MW generated")
    

In [ ]:
# Test csv_optim function
cvx_optim(cons_real_power, cons_reactive_power, max_apparent_power, min_voltage, max_voltage, BESS_nodes, wind_nodes, solar_nodes, diesel_nodes)